In [2]:

!nvidia-smi

from google.colab import drive
drive.mount('/content/drive')


Wed Feb 12 09:51:10 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   32C    P0             44W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [3]:

!pip install --upgrade pip
!pip install --upgrade transformers accelerate bitsandbytes sentencepiece
!pip install tqdm pandas


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 67.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 117.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 MB 147.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 57.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 166.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 192.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 36.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 145.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 138.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 M

In [4]:
import os
import json
import torch
import pytz
import pandas as pd
from tqdm import tqdm
from datetime import datetime

from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    AutoModelForSeq2SeqLM,
    AutoConfig,
    BitsAndBytesConfig
)


In [5]:
summarisation_file = "/content/drive/MyDrive/open_source/summarisation_text.csv"
simplification_file = "/content/drive/MyDrive/open_source/simplification_texts.csv"
gec_file = "/content/drive/MyDrive/open_source/gec_texts.csv"

output_dir = "/content/drive/MyDrive/open_source/models_results"
os.makedirs(output_dir, exist_ok=True)

print("Summarisation CSV:", summarisation_file)
print("Simplification CSV:", simplification_file)
print("GEC CSV:", gec_file)
print("Output directory:", output_dir)


Summarisation CSV: /content/drive/MyDrive/open_source/summarisation_text.csv
Simplification CSV: /content/drive/MyDrive/open_source/simplification_texts.csv
GEC CSV: /content/drive/MyDrive/open_source/gec_texts.csv
Output directory: /content/drive/MyDrive/open_source/models_results


In [6]:
models_to_run = [
    "google/flan-t5-xxl",        # T5-based Seq2Seq (~11B params)
    "google/flan-ul2",           # T5-based Seq2Seq (~20B params)
    "facebook/opt-iml-max-30b",  # CausalLM (~30B params)
    "bigscience/T0pp"            # T5-based Seq2Seq (~11B params)
]

print("Models to run:")
for m in models_to_run:
    print(" -", m)


Models to run:
 - google/flan-t5-xxl
 - google/flan-ul2
 - facebook/opt-iml-max-30b
 - bigscience/T0pp


In [7]:
# Advanced prompts for Summarization, Simplification, and GEC

####################################################
# SUMMARIZATION PROMPTS
####################################################

advanced_summarisation_prompts = [
    # 1️ Structured Stepwise Summarization (Only final summary output)
    (
        "Provide a structured summary of the following text. Follow these steps internally but output only the final summary:\n\n"
        "Step 1: Identify a suitable title based on the text.\n"
        "Step 2: Extract the three most important key points.\n"
        "Step 3: Generate a concise summary in 1-2 sentences using the key points.\n\n"
        "**Important:** Do not include the title or key points in the output. Only return the final summary.\n\n"
        "Text: [...]"
    ),

    # 2️ Simple Direct Summarization
    (
        "Summarize the following text in 2-3 sentences while keeping the key information intact.\n"
        "**Important:** Output only the final summary, without explanations or bullet points.\n\n"
        "Text: [...]"
    ),

    # 3️ Length-Controlled Summarization
    (
        "Generate a highly concise summary of the following text in a **single sentence** while preserving its main idea.\n"
        "**Return only the final summary sentence, without any additional text.**\n\n"
        "Text: [...]"
    ),
    (
        # Instruction-based Zero-shot Prompting
        """Summarisation the following Text: [...], ensuring to capture key points, highlight crucial arguments and provide a concise
        yet comprehensive understanding of the content"""
    ),
(
    # Instruction-based Zero-shot Prompting
        """Provide a summary of the following Text: [...] highlighting the key themes and arguments"""
)
]

####################################################
# SIMPLIFICATION PROMPTS
####################################################
advanced_simplification_prompts = [
    # Prompt A: Generate two different simplified versions, then pick the best
    (
        "Generate TWO different simplified versions of the text below. "
        "Then pick the one that is simpler and clearer. "
        "**Important:** Output only your final chosen version.\n\n"
        "Original text: [...]"
    ),
    # Prompt B: Two-step approach (detect problematic terms, then rewrite)
    (
        "Step 1: Identify any difficult words or phrases in the text.\n"
        "Step 2: Replace them with simpler alternatives and rewrite the text in short sentences."
        "**Important:** Provide only the simplified text.\n\n"
        "Text to simplify: [...]"
    ),
    # Iterative Refinement (Self-Reflection)
    ("Step 1: Rewrite the text below in a simpler and clearer way, while preserving its meaning.\n"
    "Step 2: Analyze your simplified text and identify any remaining complex words or unclear structures.\n"
    "Step 3: Improve the simplification by making it even more accessible. Ensure sentences are short, direct, and free of jargon.\n"
    "**Important:** Output only the final refined version.\n\n"
    "Text to simplify: [...]"),

    # Audience-Specific Simplificatio

    ("Rewrite the text below so that it is easily understandable for a 8-year-old children.\n"
    "Ensure that:\n"
    "- Sentences are short and direct.\n"
    "- Difficult words are replaced with simple alternatives.\n"
    "- The overall meaning remains the same.\n\n"
    "**Important:** Output only the final refined version.\n\n"
    "Text to simplify: [...]"

),
    # Explain & Rewrite (Chain-of-Thought Simplification)
   ("Step 1: Identify the challenging words, phrases, or sentence structures in the text and explain why they might be difficult to understand.\n"
    "Step 2: Rewrite the text in simpler language while ensuring that the original meaning is preserved.\n"
    "Step 3: Verify that the rewritten version is clear and easy to understand.\n"
    "**Important:** Provide only the final, simplified version.**\n\n"
    "Text to simplify: [...]"
)
]

####################################################
# GEC PROMPTS (Iterative / Self-Refine)
####################################################
# For grammar correction, we can use a self-refinement approach:
# 1) Make a first pass to fix errors
# 2) Double-check and refine
advanced_gec_prompts = [
    # Prompt A: Two-step correction (correct, then refine)
    (
        "Perform a two-step grammar correction:\n"
        "1) Correct all obvious grammar/spelling mistakes.\n"
        "2) Re-check the sentence and refine if needed.\n"
       "**Important:** Only return the final corrected version..\n\n"
        "Input sentence: [...]"
    ),
    # Prompt B: Another two-pass approach with emphasis on fluency
    (
        "First pass: fix grammatical errors.\n"
        "Second pass: ensure the sentence is fluent and natural in standard English.\n"
        "**Important:** Return the final corrected version only.\n\n"
        "Sentence: [...]"
    ),
   (
      "Step 1: Identify and list any grammar, spelling, or fluency errors in the sentence.\n"
      "Step 2: Explain why each mistake is incorrect and how it can be improved.\n"
      "Step 3: Rewrite the corrected sentence in standard English with natural fluency.\n"
      "**Important:** Return only the final corrected version.\n\n, without any additional text.\n\n"
      "Sentence: [...]"),

   ( "Perform a multi-level correction of the following sentence:\n"
"1) Fix all grammatical, spelling, and punctuation mistakes.\n"
"2) Improve clarity by simplifying complex structures.\n"
"3) Enhance precision by removing unnecessary words or ambiguity.\n"
"**Important:** Provide only the final improved version.\n\n"
"Sentence: [...]")


]

####################################################
# TEMPERATURE SETTINGS
####################################################
temperatures = [0.01, 0.5, 0.7]

print("Loaded advanced prompting strategies.")
print("Temperatures:", temperatures)


Loaded advanced prompting strategies.
Temperatures: [0.01, 0.5, 0.7]


In [8]:

task_details = {
    "Summarisation": {
        "csv_path": summarisation_file,
        "prompts": advanced_summarisation_prompts
    },
    "Simplification": {
        "csv_path": simplification_file,
        "prompts": advanced_simplification_prompts
    },
    "GEC": {
        "csv_path": gec_file,
        "prompts": advanced_gec_prompts
    }
}

tasks_to_run = ["Summarisation", "Simplification", "GEC"]

time_now = datetime.now(pytz.timezone('Europe/London')).strftime('%Y-%m-%dT%H:%M:%S')

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)

print("Tasks to run:", tasks_to_run)
print("Current time for output naming:", time_now)


Tasks to run: ['Summarisation', 'Simplification', 'GEC']
Current time for output naming: 2025-02-12T09:53:03


In [ ]:
# Main loop for tasks
for task in tasks_to_run:
    print(f"\n=== Starting Task: {task} ===")

    # Load the CSV file for this task
    csv_path = task_details[task]["csv_path"]
    if not os.path.exists(csv_path):
        print(f"CSV file for task {task} not found at: {csv_path}")
        continue

    df_task = pd.read_csv(csv_path)
    inputs = df_task["displayed_text"].tolist()
    print(f"Loaded CSV: {csv_path}, number of texts: {len(inputs)}")

    # Get the prompts for this task
    prompts = task_details[task]["prompts"]

    # Prepare a results dictionary for THIS specific task
    results = {
        "model": [],
        "prompt": [],
        "input": [],
        "detokenized_input": [],
        "original_input": [],
        "output": [],
        "temperature": []
    }

    # Loop through models for the current task
    for model_name in models_to_run:
        print(f"\n--- Inference with model: {model_name} for Task: {task} ---")

        try:
            # Load the model configuration and tokenizer
            config = AutoConfig.from_pretrained(model_name)
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            model_type = config.model_type

            # Force everything on GPU0
            device_map = {"": 0}

            # Load model in 4-Bit
            if model_type in ["t5", "mt5"]:
                # T5 => Seq2Seq
                model = AutoModelForSeq2SeqLM.from_pretrained(
                    model_name,
                    quantization_config=bnb_config,
                    device_map=device_map
                )
            else:
                # GPT/OPT => CausalLM
                model = AutoModelForCausalLM.from_pretrained(
                    model_name,
                    quantization_config=bnb_config,
                    device_map=device_map
                )
        except Exception as e:
            print(f"Error loading model {model_name}: {e}")
            continue

        # Go through each input text
        for text_input in tqdm(inputs, desc=f"{model_name} on {task}"):
            for prompt in prompts:
                for temperature in temperatures:
                    try:
                        # Insert the text into the prompt
                        text = prompt.replace("[...]", text_input)

                        # Store metadata
                        results["model"].append(model_name)
                        results["prompt"].append(prompt)
                        results["temperature"].append(temperature)
                        results["input"].append(text)
                        results["original_input"].append(text_input)

                        # Detokenize the prompt for reference
                        detokenized_text = tokenizer.decode(
                            tokenizer.encode(text),
                            skip_special_tokens=True
                        )
                        results["detokenized_input"].append(detokenized_text)

                        # If it's OPT, we allow more length
                        gen_max_length = 256 if model_type == "opt" else 128

                        # Encode input on GPU
                        input_ids = tokenizer.encode(
                            text,
                            return_tensors="pt",
                            truncation=True,
                            max_length=128
                        ).to("cuda")

                        # Generate
                        output_ids = model.generate(
                            input_ids,
                            max_length=gen_max_length,
                            temperature=temperature,
                            do_sample=True
                        )

                        # Decode result
                        gen_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
                        results["output"].append(gen_text)
                    except Exception as e:
                        print(f"Error during inference for input '{text_input}': {e}")

        # Unload model from GPU
        del model
        torch.cuda.empty_cache()

    # Convert results dictionary to DataFrame
    if len(results["model"]) == 0:
        print(f"No results to save for task: {task}")
        continue

    df_results = pd.DataFrame.from_dict(results)

    # Build output filenames
    csv_path_out = os.path.join(output_dir, f"output_{task.lower()}_{time_now}.csv")
    json_path_out = os.path.join(output_dir, f"output_{task.lower()}_{time_now}.json")

    try:
        # Save results as CSV
        df_results.to_csv(csv_path_out, index=False, encoding="utf-8")
        print(f"Saved CSV => {csv_path_out}")

        # Save results as JSON
        with open(json_path_out, "w", encoding="utf-8") as f_js:
            json.dump(results, f_js, ensure_ascii=False)
        print(f"Saved JSON => {json_path_out}")

    except Exception as e:
        print(f"Error saving results for task {task}: {e}")

    print(f"\nTask {task} finished!")

print("\n==== All Tasks Completed! ====")
print("Check your output files in:", output_dir)



=== Starting Task: Summarisation ===
Loaded CSV: /content/drive/MyDrive/open_source/summarisation_text.csv, number of texts: 100

--- Inference with model: google/flan-t5-xxl for Task: Summarisation ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/674 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

...

Loading checkpoint shards:   0%|          | 0/7 [00:00<?, ?it/s]

facebook/opt-iml-max-30b on GEC: 100%|██████████| 100/100 [47:54<00:00, 28.74s/it]



--- Inference with model: bigscience/T0pp for Task: GEC ---


bigscience/T0pp on GEC: 100%|██████████| 100/100 [28:45<00:00, 17.26s/it]


Saved CSV => /content/drive/MyDrive/open_source/models_results/output_gec_2025-02-08T14:54:33.csv
Saved JSON => /content/drive/MyDrive/open_source/models_results/output_gec_2025-02-08T14:54:33.json

Task GEC finished!

==== All Tasks Completed! ====
Check your output files in: /content/drive/MyDrive/open_source/models_results
